# Récupération/Traitement des données de formes d'ondes

> Ce Notebook présente la récupération des formes d'ondes pour le calcul des PPSDs et du déplacement RMS.

---
## Importation des modules

In [1]:
import glob, os
import numpy as np
import pandas as pd
import warnings
import datetime
import sys
from obspy import UTCDateTime, read
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.client import FDSNNoDataException
from obspy.signal import PPSD
from seismic_noise import download_noise, process_PPSD, load_PPSD, process_DRMS
import urllib.request
import zipfile

---
## Paramètre initiaux

In [2]:
time_zone = "Europe/Brussels"      # Format d'heure
dl_noise = True                   # Téléchargement des données
dl_PPSD = False                     # Téléchargement des PPSDs
proc_PPSD = True                  # Calcul des PPSDs
proc_DRMS = True                  # Calcul du déplacement RMS
delete_MSEED = False               # Suppression des fichiers MSEED
PPSD_FOLDER  = "C:/Users/Arnaud/Desktop/Travail/Projet de Recherche/SeismicNoiseData/PPSD"
DRMS_FOLDER  = "C:/Users/Arnaud/Desktop/Travail/Projet de Recherche/SeismicNoiseData/DRMS"
MSEED_FOLDER = "C:/Users/Arnaud/Desktop/Travail/Projet de Recherche/SeismicNoiseData/MSEED"
freqs = [(2, 8)]       # Bandes de fréquence à étudier
start_date = "2010-01-01"         # Date de début
end_date = "2020-12-31"           # Date de fin
st_file = "DATA/st_metadata/stations_antarctic.txt"  # Fichier de stations
# all_stations = ["FR.STR.00.HHZ"]

---
## Processing des formes d'ondes

**Préparation des répertoires**

In [3]:
start = UTCDateTime(start_date)
end = UTCDateTime(end_date) + 86340

### Liste des stations à traiter
try:
    # Si all_stations a été initialisé avant, on passe
    type(all_stations)
except:
    # Si all_stations n'a pas été initialisé avant, on récupère le fichier contenant les stations
    all_stations = np.loadtxt(st_file, dtype=str)
    try:
        len(all_stations)
    except:
        all_stations = np.append(list(), all_stations)
    
### Création des répertoires de données
if not os.path.exists(MSEED_FOLDER): os.makedirs(MSEED_FOLDER)
if not os.path.exists(PPSD_FOLDER):  os.makedirs(PPSD_FOLDER)
if not os.path.exists(DRMS_FOLDER):  os.makedirs(DRMS_FOLDER)
    
period = [start, end]

**Processing des données**

In [ ]:
print("[{}] [INFO] Début du programme".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
datelist = pd.date_range(start.datetime, min(end, UTCDateTime()).datetime, freq="D")  # Jours à récupérer
client = Client("RESIF")  # Client utilisé pour récupérer les donnée via le webservice

k = 1
for station_str in all_stations:
    
    PPSD_FILES  = PPSD_FOLDER  + "/{}/".format(station_str)
    DRMS_FILES  = DRMS_FOLDER  + "/{}/".format(station_str)
    MSEED_FILES = MSEED_FOLDER + "/{}/".format(station_str)
    print("[{}] [INFO] Station {}/{} {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), k, len(all_stations), station_str))
    
    ### Téléchargement des données
    if dl_noise:
        print("[{}] [INFO]     Collecting ...".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        download_noise(start, end, time_zone, station_str, client, MSEED_FILES, datelist)
    
    ### Téléchargement des PPSDs
    if dl_PPSD:
        print("[{}] [INFO]     Collecting PPSDs ... ".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        filename = PPSD_FOLDER + '/' + station_str + '/data.zip'
        if not os.path.exists(PPSD_FOLDER + '/' + station_str):  os.makedirs(PPSD_FOLDER + '/' + station_str)
        net, sta, loc, cha = tuple(station_str.split("."))
        url = "https://ws.resif.fr/resifws/ppsd/1/query?net={}&sta={}&loc={}&cha={}&starttime={}&endtime={}&format=npz&nodata=404".format(net,sta,loc,cha,start_date,end_date)
        try:
            urllib.request.urlretrieve(url, filename)
        except:
            pass
        else:
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall(PPSD_FOLDER + '/' + station_str)
            os.remove(filename)
    
    ### Calcul des PPSDs
    if proc_PPSD:
        print("[{}] [INFO]     Processing PPSDs ... ".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        ppsds = process_PPSD(start, end, time_zone, station_str, client, MSEED_FILES, PPSD_FILES, datelist)
        if delete_MSEED:
            for f in glob.glob("{}*.mseed".format(MSEED_FILES)):
                try:
                    os.remove(f)
                except:
                    pass
    
    ### Calcul du déplacement RMS
    if proc_DRMS:
        print("[{}] [INFO]     Processing RMS ... ".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        ppsds = load_PPSD(np.append(list(), station_str), period, PPSD_FILES)
        process_DRMS(ppsds, freqs, DRMS_FOLDER)
        
    k += 1

print("[{}] [INFO] FIN - Arrêt du programme".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))

[2021-09-13 18:23:29] [INFO] Début du programme
[2021-09-13 18:23:29] [INFO] Station 1/2 G.CCD.00.BHZ
[2021-09-13 18:23:29] [INFO]     Collecting ...
[2021-09-13 18:29:15] [INFO]     Processing PPSDs ... 


C:\Users\Arnaud\anaconda3\envs\obspy\lib\site-packages\obspy\io\mseed\headers.py:828: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 24576 to 24703.
  warnings.warn(_w, InternalMSEEDWarning)
C:\Users\Arnaud\anaconda3\envs\obspy\lib\site-packages\obspy\io\mseed\headers.py:828: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 24704 to 24831.
  warnings.warn(_w, InternalMSEEDWarning)
C:\Users\Arnaud\anaconda3\envs\obspy\lib\site-packages\obspy\io\mseed\headers.py:828: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 24832 to 24959.
  warnings.warn(_w, InternalMSEEDWarning)
C:\Users\Arnaud\anaconda3\envs\obspy\lib\site-packages\obspy\io\mseed\headers.py:828: InternalMSEEDWarning: readMSEEDBuffer(): Not a SEED record. Will skip bytes 24960 to 25087.
  warnings.warn(_w, InternalMSEEDWarning)
C:\Users\Arnaud\anaconda3\envs\obspy\lib\site-packages\obspy\io\mseed\headers.py:828: InternalMSEEDWarning: readMSEEDBuf